In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from scipy.stats import norm
from plotly.subplots import make_subplots
from scipy import stats
from datetime import date,datetime
from util import date_diff

In [2]:
df = pd.read_excel('site_info_ver_2.3.xlsx')

In [6]:
sites = df.Site_Name.unique()

In [3]:
global_no_heat_wave = []
global_heat_wave_week = []
global_israelis_heat= []
global_tourists_heat = []
global_israelis_no_heat = []
global_tourists_no_heat = []
dates_no_heat_wave = []
dates_heat_wave_week = []
dates_israelis_heat= []
dates_tourists_heat = []
dates_israelis_no_heat = []
dates_tourists_no_heat = []

In [4]:
def test(tmp):
    no_heat_wave = []
    heat_wave_week = []
    israelis_heat= []
    tourists_heat = []
    israelis_no_heat = []
    tourists_no_heat = []


    for i in range(8,tmp.shape[0]-7):
        if tmp.iloc[i,3]==1:
            
                res = tmp.iloc[i-7:i,3].sum() #check prev week that all 0's -> no heat wave
                delta0 = date_diff(tmp.iloc[i+7,0],tmp.iloc[i,0],7)
                delta1 = date_diff(tmp.iloc[i-1,0] , tmp.iloc[i-8,0],7)
                if res == 0 and delta0 and delta1: 

                    for index in range(i,i+7):
                        heat_wave_week.append(tmp.iloc[index,4])
                        israelis_heat.append(tmp.iloc[index,1])
                        tourists_heat.append(tmp.iloc[index,2])
                        global_heat_wave_week.append(tmp.iloc[index,4])
                        global_israelis_heat.append(tmp.iloc[index,1])
                        global_tourists_heat.append(tmp.iloc[index,2])
                        dates_heat_wave_week.append(tmp.iloc[index,0])
                        dates_israelis_heat.append(tmp.iloc[index,0])
                        dates_tourists_heat.append(tmp.iloc[index,0])

                    for index in range(i-8,i-1):
                        no_heat_wave.append(tmp.iloc[index,4])
                        israelis_no_heat.append(tmp.iloc[index,1])
                        tourists_no_heat.append(tmp.iloc[index,2])
                        global_no_heat_wave.append(tmp.iloc[index,4])
                        global_israelis_no_heat.append(tmp.iloc[index,1])
                        global_tourists_no_heat.append(tmp.iloc[index,2])
                        dates_no_heat_wave.append(tmp.iloc[index,0])
                        dates_israelis_no_heat.append(tmp.iloc[index,0])
                        dates_tourists_no_heat.append(tmp.iloc[index,0])

    if len(heat_wave_week)>0 and len(no_heat_wave)>0 and len(israelis_heat)>0 and len(israelis_no_heat)>0 and len(tourists_heat)>0 and len(tourists_no_heat)>0:       

        output = [heat_wave_week,no_heat_wave,israelis_heat,israelis_no_heat,tourists_heat,tourists_no_heat]  
        # print("total no heat : ",no_heat_wave)
        # print("total heat",heat_wave_week)
        res = [] 
        #compare total
        stat, p = stats.wilcoxon(no_heat_wave,heat_wave_week,alternative="greater",zero_method='wilcox')
        res.append(p)
        # print("israelis no heat",israelis_no_heat)
        # print("israelis heat",israelis_heat)
        #compare israelis in heat wave vs. israelis in no heat wave
        stat, p = stats.wilcoxon(israelis_no_heat,israelis_heat,alternative="greater",zero_method='wilcox')
        res.append(p)
        # print("tourists heat",tourists_heat)
        # print("tourists no heat",tourists_no_heat)
        #compare tourists in heat wave vs. tourists in no heat wave
        try:
            stat, p = stats.wilcoxon(tourists_no_heat,tourists_heat,alternative="greater",zero_method='wilcox')
            res.append(p)
        except: return res,list(output)
        return res,list(output)
    else: return None,None
        




In [7]:

# classifier = ['Total heat : ','Total no heat: ','Israelis heat: ','Israelis no heat: ','Tourists heat: ','Tourists no heat: ']
with open('dog_breeds_reversed.txt', 'a') as writer:
    for site in sites:
        tmp = df[df.Site_Name == site]
        tmp = tmp[['Date','Israelis_Count','Tourists_Count','is_HeatWave','Total']].reset_index(drop = True)
        result,vars = test(tmp)
        if result is not None and len(result) > 0:
            writer.write('------------------'+site+'-------------------\n')
            print('------------------'+site+'-------------------\n')
            for i in range(len(result)):
                pval = result[i]
                if i == 0 :
                    if pval < 0.05:
                        writer.write("there was a diff in total entries when there was a heatwave " + str(pval)+"\n")
                        #print("there was a diff in total entries when there was a heatwave " + str(pval))
                    else :
                        writer.write("there was not a diff in total entries when there was a heatwave "+ str(pval)+"\n")
                        #print("there was not a diff in total entries when there was a heatwave "+ str(pval))
                
                elif i == 1:
                    if pval < 0.05:
                        writer.write("there was a diff in total israelis when there was a heatwave " + str(pval)+"\n")
                        #print("there was a diff in total israelis when there was a heatwave " + str(pval))
                    else :
                        writer.write("there was not diff in total israelis when there was a heatwave "+ str(pval)+"\n")
                        #print("there was not diff in total israelis when there was a heatwave "+ str(pval))
                
                else :
                    if pval < 0.05:
                        writer.write("there was a diff in total tourists when there was a heatwave " + str(pval)+"\n")
                        #print("there was a diff in total tourists when there was a heatwave " + str(pval))
                    else :
                        writer.write("there was not diff in total tourists when there was a heatwave "+ str(pval)+"\n")
                        #print("there was not diff in total tourists when there was a heatwave "+ str(pval))
            for x in vars:
                print(x)
            writer.write('--------------------------------------------\n')
            print('--------------------------------------------\n')

writer.close()



------------------Apollonia-------------------

[74, 323, 74, 183, 12, 30, 81, 207, 57, 8, 45, 33, 62, 254, 94, 88, 20, 66, 29, 30, 30, 47, 21, 72, 25, 45, 427, 145, 82, 26, 61, 23, 96, 292, 41, 382, 25, 55, 16, 61, 42, 116, 514, 22, 212, 514, 15, 9, 120, 140, 22, 34, 17, 41, 143, 73, 28, 49, 94, 31, 100, 503, 83, 40, 28, 76, 233, 63, 19, 48, 57, 86, 105, 36, 56, 29, 44, 104, 121, 99, 221, 72, 218, 76, 431, 161, 609, 80, 194, 36, 83, 242, 101, 85, 32, 114, 1300, 65, 98, 12, 14, 359, 45, 76, 54, 284, 41, 84, 129, 59, 67, 104, 515, 24, 19, 66, 548, 55, 133, 71, 89, 69, 22, 137, 116, 1106, 614, 433, 17, 11, 5, 11, 49, 719, 172, 451, 208, 231, 325, 184, 455, 1043, 55, 13, 37, 74, 145, 111, 304, 93, 78, 24, 144, 204]
[88, 114, 865, 207, 34, 89, 154, 238, 916, 149, 95, 19, 26, 1125, 154, 110, 718, 22, 37, 62, 3, 20, 66, 29, 30, 30, 39, 385, 21, 110, 59, 25, 25, 61, 230, 22, 412, 20, 38, 70, 52, 150, 60, 558, 80, 29, 22, 50, 32, 133, 161, 29, 86, 21, 24, 16, 195, 144, 63, 20, 49, 102, 1384, 7

C:\Users\liavc\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
C:\Users\liavc\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\morestats.py:3155: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


------------------Baram-------------------

[62, 34, 50, 320, 27, 111, 37, 97, 15, 16, 38, 40, 11, 36, 20, 4, 44, 64, 6, 104, 17, 58, 24, 17, 55, 15, 33, 62, 22, 18, 30, 69, 14, 14, 15, 31, 30, 11, 13, 72, 8, 31, 41, 14, 65, 63, 13, 13, 51, 215, 94, 15, 87, 6, 24, 19, 47, 32, 67, 46, 10, 111, 32, 5, 18, 93, 71, 9, 24, 181, 177, 165, 101, 108, 102, 11, 72, 8, 89, 35, 43, 37, 29, 32, 55, 50, 19, 64, 19, 54, 24, 57, 16, 48, 27, 19, 3, 131, 46, 60, 42, 197, 12, 1, 27, 4, 54, 145, 35, 9, 22, 136, 17, 24, 30, 74, 7, 11, 66, 9, 63, 35, 28, 43, 79, 13, 23, 36, 25, 90, 39, 40, 69, 42, 16, 72, 59, 119, 29, 7, 56, 146, 43, 64, 160, 245, 104, 54, 13, 47, 19, 56, 13, 10, 32, 64, 132, 81, 16, 24, 42]
[108, 112, 16, 4, 28, 9, 181, 105, 192, 25, 10, 14, 100, 38, 11, 36, 20, 139, 35, 13, 81, 14, 57, 52, 74, 27, 9, 63, 18, 42, 21, 91, 5, 11, 81, 19, 15, 28, 56, 15, 13, 4, 12, 39, 2, 200, 35, 56, 20, 21, 65, 33, 92, 121, 17, 8, 89, 44, 14, 29, 19, 20, 131, 64, 17, 15, 65, 10, 12, 44, 194, 187, 102, 110, 

In [12]:
stat, p = stats.wilcoxon(global_no_heat_wave,global_heat_wave_week,zero_method='wilcox',alternative="greater")

if p < 0.05:
    # print("There was a difference: reject H0 ",str(p))
    print("No heat Wave is Greater: reject H0 ",str(round(p,50)))
else: print("There was no difference: accept H0 ",str(p))

No heat Wave is Greater: reject H0  3.1217078042582066e-07


In [27]:
no_heat_south = []
heat_wave_south = []
heat_wave_central = []
no_heat_central = []
heat_wave_north = []
no_heat_north = []

In [28]:
def test_regions(tmp):

    for i in range(8,tmp.shape[0]-7):
        if tmp.iloc[i,3]==1 and tmp.iloc[i,5] == 1: #5 = central region
            res = tmp.iloc[i-7:i,3].sum() #check prev week that all 0's -> no heat wave
            delta0 = date_diff(tmp.iloc[i+7,0],tmp.iloc[i,0],7)
            delta1 = date_diff(tmp.iloc[i-1,0] , tmp.iloc[i-8,0],7)
            if res == 0 and delta0 and delta1: 
                for index in range(i,i+7):
                    heat_wave_central.append(tmp.iloc[index,4])

                for index in range(i-8,i-1):
                    no_heat_central.append(tmp.iloc[index,4])
                    
    for i in range(8,tmp.shape[0]-7):
        if tmp.iloc[i,3]==1 and tmp.iloc[i,6] == 1: #6 = south region
            res = tmp.iloc[i-7:i,3].sum() #check prev week that all 0's -> no heat wave
            delta0 = date_diff(tmp.iloc[i+7,0],tmp.iloc[i,0],7)
            delta1 = date_diff(tmp.iloc[i-1,0] , tmp.iloc[i-8,0],7)
            if res == 0 and delta0 and delta1: 
                for index in range(i,i+7):
                    heat_wave_south.append(tmp.iloc[index,4])
                for index in range(i-8,i-1):
                    no_heat_south.append(tmp.iloc[index,4])


    for i in range(8,tmp.shape[0]-7):
        if tmp.iloc[i,3]==1 and tmp.iloc[i,7] == 1: #7 = north region
            res = tmp.iloc[i-7:i,3].sum() #check prev week that all 0's -> no heat wave
            delta0 = date_diff(tmp.iloc[i+7,0],tmp.iloc[i,0],7)
            delta1 = date_diff(tmp.iloc[i-1,0] , tmp.iloc[i-8,0],7)
            if res == 0 and delta0 and delta1: 
                for index in range(i,i+7):
                    heat_wave_north.append(tmp.iloc[index,4])
                for index in range(i-8,i-1):
                    no_heat_north.append(tmp.iloc[index,4])
   

In [29]:
for site in sites:
    tmp = df[df.Site_Name == site]
    tmp = tmp[['Date','Israelis_Count','Tourists_Count','is_HeatWave','Total','region_Central','region_South','region_North']].reset_index(drop = True)
    test_regions(tmp)

In [39]:
#check north region vs south region on heat wave
south = list(df.loc[(df.region_South == 1) & (df.is_HeatWave == 1)]['Total'])
north = list(df.loc[(df.region_North == 1) & (df.is_HeatWave == 1)]['Total'])

#are there more visits in the south when there is a heat wave than in the north ? 
stat, p = stats.mannwhitneyu(south,north,alternative="greater")

if p < 0.05:
    # print("There was a difference: reject H0 ",str(p))
    print("No heat Wave is Greater: reject H0 ",str(p))
else: print("There was no difference: accept H0 ",str(p))


There was no difference: accept H0  1.0


In [30]:
#check north region
stat, p = stats.wilcoxon(no_heat_north,heat_wave_north,zero_method='wilcox',alternative="greater")

if p < 0.05:
    # print("There was a difference: reject H0 ",str(p))
    print("No heat Wave is Greater: reject H0 ",str(p))
else: print("There was no difference: accept H0 ",str(p))

No heat Wave is Greater: reject H0  0.014122026534673957


In [25]:
#check central region
stat, p = stats.wilcoxon(no_heat_central,heat_wave_central,zero_method='wilcox',alternative="greater")

if p < 0.05:
    # print("There was a difference: reject H0 ",str(p))
    print("No heat Wave is Greater: reject H0 ",str(p))
else: print("There was no difference: accept H0 ",str(p))

No heat Wave is Greater: reject H0  0.0002823667520529224


In [26]:
#check south region
stat, p = stats.wilcoxon(no_heat_south,heat_wave_south,zero_method='wilcox',alternative="greater")

if p < 0.05:
    # print("There was a difference: reject H0 ",str(p))
    print("No heat Wave is Greater: reject H0 ",str(p))
else: print("There was no difference: accept H0 ",str(p))

No heat Wave is Greater: reject H0  9.135433446452378e-06


In [57]:
stat, p = stats.wilcoxon(global_no_heat_wave,global_heat_wave_week,zero_method='wilcox',alternative="greater")

if p < 0.05:
    # print("There was a difference: reject H0 ",str(p))
    print("No heat Wave is Greater: reject H0 ",str(p))
else: print("There was no difference: accept H0 ",str(p))

No heat Wave is Greater: reject H0  3.6806746736290164e-11


In [59]:
stat, p = stats.wilcoxon(global_israelis_no_heat,global_israelis_heat,zero_method='wilcox',alternative="greater")

if p < 0.05:
    print("There was a difference: reject H0 ",str(p))
else: print("There was no difference: accept H0 ",str(p))

There was a difference: reject H0  5.403965987061008e-09


In [60]:
stat, p = stats.wilcoxon(global_tourists_no_heat,global_tourists_heat,zero_method='wilcox',alternative="greater")

if p < 0.05:
    print("There was a difference: reject H0 ",str(p))
else: print("There was no difference: accept H0 ",str(p))

There was a difference: reject H0  0.0001863051123023141


In [ ]:
# total_heat_df = pd.DataFrame(data=[dates_heat_wave_week,global_heat_wave_week,dates_no_heat_wave,global_no_heat_wave])
# total_heat_df = total_heat_df.T
# total_heat_df.columns=['date_Heat','total_Heat','date_No_Heat','total_No_Heat']

# total_heat_df.head(7)

In [68]:
# total_heat_df = pd.DataFrame(data=[dates_heat_wave_week,global_heat_wave_week,dates_no_heat_wave,global_no_heat_wave])
# total_heat_df = total_heat_df.T
# total_heat_df.columns=['date_Heat','total_Heat','date_No_Heat','total_No_Heat']
# # total_heat_df.sort_values('date',inplace= True)
# total_heat_df.to_excel('total_df.xlsx')

In [69]:
# israelis_heat_df = pd.DataFrame(data=[dates_israelis_heat,global_israelis_heat,dates_israelis_no_heat,global_israelis_no_heat])
# israelis_heat_df = israelis_heat_df.T
# israelis_heat_df.columns=['date_Heat','israelis_Heat','date_No_Heat','israelis_No_Heat']
# # israelis_heat_df.sort_values('date',inplace= True)
# israelis_heat_df.to_excel('israelis_df.xlsx')


# tourists_heat_df = pd.DataFrame(data=[dates_tourists_heat,global_tourists_heat,dates_tourists_no_heat,global_tourists_no_heat])
# tourists_heat_df = tourists_heat_df.T
# tourists_heat_df.columns=['date_Heat','tourists_Heat','date_No_Heat','tourists_No_Heat']
# # tourists_heat_df.sort_values('date',inplace= True)
# tourists_heat_df.to_excel('tourists_df.xlsx')

